# 📌ACTIVIDAD 3: CREAR UN SISTEMA TASADOR DE VIVIENDAS.

### DEFINIR Y RECOPILAR DATOS
En primer lugar debemos ponernos de acuerdo de manera que cada uno de nosotros piense una
característica que: <br><br>
• Pueda influir en el precio de una vivienda.<br><br>
• Sea una información que tengamos la capacidad de recolectar.<br><br>
ENTREGA 5: Añade al documento enlace enlace: <br><br>
a) En la hoja de cálculo "datos_viviendas.xlsl" en la hoja datos, añade una columna cuyo
prefijo tenga tus iniciales. Ya tiene estas nueve: josrosrod_autor, josrosrod_origen,
josrosrod_municipio, josrosrod_dia, josrosrod_mes, josrosrod_año, josrosrod_lat,
josrosrod_lon, josrosrod_precio<br><br>
b) En la hoja "descripción" añade una descripción del significado de esa columna y de cómo se
puede conseguir. Ya tendrás anotaciones de las características que están añadidas.<br><br>
Cuando ya tengamos definidas las características que vamos a utilizar para esta actividad, vamos a
recolectar datos, al menos 10 cada uno. Intentando completar la información que no dispongamos
(intentaremos no inventar nada, de manera que cuando algo no lo sepamos, tendremos que indicar de
donde lo hemos conseguido). No puedes dejar más de un valor ausente en tus datos.

ENTREGA: <br><br>
a0) Añade a la hoja de cálculo 10 ejemplos y aporta una columna.<br><br>
a) Análisis y preprocesamiento de datos:<br><br>
• Análisis de datos: histogramas, boxplots, cantidad de datos ausentes, presencia de
outliers.<br><br>
• Codificación de categóricas: label-encoding o one-hot-encoding.<br><br>
• Detección e imputación de ausentes.<br><br>
• Detección e imputación de anomalías.<br><br>
• Escalado de numéricas.<br><br>
• Debes ir creando un pipeline de manera que se le facilite uno de los ejemplos y lo
preprocese para dárselo al modelo.<br><br>
a) Selección de características:<br><br>
• Matriz de correlaciones y mapa de calor: más fácil de interpretar si dejas el target como
la primera característica.<br><br>
• Estudio estadístico univariado (eliminar características con poca influencia en target).<br><br>
• Estudio de colinealidad por parejas (eliminar alta correlación entre predictoras).<br><br>
c) Selección de modelos de regresión: debes considerar al menos 5 diferentes y uno de ellos el
método de mínimos cuadrados. Recuerda que tienes LinearRegressor, SGDRegressor, Ridge,
Lasso, ElasticNet, K-NN, polinómico...<br><br>
d) Estudio de varianza de modelos con CV.<br><br>
e) Seleccionar los dos que mejor desempeño muestren justificando su elección.<br><br>
f) Configuración de hiperparámetros con grid-search en los mejores 2 modelos seleccionados.<br><br>
f) Estudio final de errores en las predicciones y gráfico de importancia de características.<br><br>
g) Selección justificada del mejor y su descripción:<br><br>
• Qué predictoras utiliza.<br><br>
• Una descripción de su utilización y procesos que realiza (preprocesamiento, ...).<br><br>
• Qué desempeño alcanza en train y test (indica también la métrica empleada)<br><br>
• Una valoración final de tu modelo según los resultados obtenidos.<br><br>
h) Guarda el modelo en un fichero con el nombre <iniciales>_modelo_u02_p03_a3.joblib
donde <iniciales> son las 3 primeras letras del nombre y apellidos, en mi caso sería
"josrosrod_modelo_u02_p03_a3.joblib".<br><br>
La entrega de esta última actividad es el 70% de la nota de la práctica y se valora:<br><br>
• 50% puntos por la entrega: la realización de las actividades propuestas.<br><br>
• Al modelo que entregues se le pedirá que realice 10 predicciones con datos probablemente no
vistos. Según el error promedio que cometa se le asignará posición en un ranking. En función
de la posición que ocupe (lo bien que lo haga), conseguirá más o menos puntuación adicional
hasta completar el 20% restante
